In [50]:
from pathlib import Path
import torch
import wandb
api = wandb.Api()
from eval.cls.model import Generator
import h5py
import importlib


import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
from PIL import Image
from tqdm import tqdm




In [2]:
m = importlib.import_module('utils.datasets')

In [11]:
dataset = getattr(m, "CIFAR10_LT")(is_train=True, is_extension=False)

num_make_data = dataset.num_classes.max() - dataset.num_classes

num_make_data = np.hstack([np.ones(n)*c for c, n in enumerate(num_make_data)]).astype(np.int32)
num_make_data
# np.unique(num_make_data, return_counts=True)
# num_make_data.size
# for c, n in enumerate(num_make_data):
    # print(c, n)
    
    
    
    


array([1, 1, 1, ..., 9, 9, 9], dtype=int32)

In [25]:
num_make_data

temp = []
for i in range(0, len(num_make_data), 128):
    temp.append(num_make_data[i: i+128])
    # print(i, i+128)
    
temp = np.hstack(temp)
print(np.unique(temp, return_counts=True))

(array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), array([4900, 4800, 4700, 4600, 4500, 4400, 4300, 4200, 4100]))


In [59]:
x = torch.tensor([1, 2, 3])[None,...]
print(x.size())
print(x.repeat(4,1).size())
x.repeat(4, 2, 1).size()

torch.Size([1, 3])


RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor

In [60]:
runs = api.runs('MYTEST0')
for run in runs:    
    # if 'ECOGAN' in i.name:
    # if 'MNIST' in i.config['data_name']:
    #     t=i
    #     print(i)
    # print(i.n÷ame ,i.config['data_name'])
    
    id = run.id
    data_name = run.config['data_name']
    if data_name == 'imb_CIFAR10': data_name="CIFAR10_LT"
    if data_name == 'imb_FashionMNIST': data_name="FashionMNIST_LT"
    if data_name == 'imb_MNIST': continue
    if data_name == 'Places_LT': continue
    print(id, data_name)
    
    dataset = getattr(m, data_name)(is_train=True, is_extension=False)
    
    artifact_dir = run.logged_artifacts()[0].download()
    ch = torch.load(Path(artifact_dir) / "model.ckpt")
    state_dict = {'.'.join(k.split('.')[1:]): w for k, w in ch['state_dict'].items() if 'G' in k}
    
    G = Generator(**ch['hyper_parameters']).cuda()
    G.load_state_dict(state_dict)
    G.eval()
    
    num_make_data = dataset.num_classes.max() - dataset.num_classes
    num_make_data = np.hstack([np.ones(n)*c for c, n in enumerate(num_make_data)]).astype(np.int32)
    
    # print(num_make_data)
    w, h, c = 64, 64, 3
    
    with h5py.File('gen.h5py', 'w') as file:
        d_name = f'gen/{id}/{data_name}/data'
        data_dataset = file.create_dataset(d_name, shape=(len(num_make_data), w, h, c))
        t_name = f'gen/{id}/{data_name}/targets'
        targets_dataset = file.create_dataset(t_name, shape=(len(num_make_data),))
    
        for i, c in enumerate(tqdm(num_make_data)):
            img = G(torch.rand(1, 128).cuda(), torch.tensor([c]).cuda())
            img = img[0]
            if img.size(0) == 1:
                img = img.repeat(3, 1, 1)
            img = to_pil_image(img*0.5+0.5)
            img = np.array(img)
            data_dataset[i] = img
            targets_dataset[i] = c
            
            
            # plt.imshow(img,  cmap='gray')
            # plt.show()


1r08m7xe CIFAR10_LT


wandb: Downloading large artifact model-1r08m7xe:v0, 157.30MB. 1 files... Done. 0:0:0.1
100%|██████████| 40500/40500 [01:19<00:00, 508.75it/s]


3p2vbr49 FashionMNIST_LT


wandb: Downloading large artifact model-3p2vbr49:v0, 157.26MB. 1 files... Done. 0:0:0.1
  0%|          | 0/49500 [00:00<?, ?it/s]


TypeError: Can't broadcast (64, 64) -> (64, 64, 3)

In [65]:
img = G(torch.rand(1, 128).cuda(), torch.tensor([c]).cuda())
img = img
print(img.size())


torch.Size([1, 1, 64, 64])


1

In [64]:
print(img.min(), img.max(), img.mean(), img.std())

tensor(-0.2500, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.5961, device='cuda:0', grad_fn=<MaxBackward1>) tensor(-0.1170, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.2379, device='cuda:0', grad_fn=<StdBackward0>)


In [97]:
G.eval()
with torch.no_grad():
    img = G(torch.rand(4000, 128).cuda(), torch.tensor([0]).cuda())


# img = to_pil_image(img[0]*0.5+0.5)
for i in range((img[0]*0.5+0.5).size(0)): 
    print(torch.tensor(np.array(img[i]), dtype=torch.float).max(),
        torch.tensor(np.array(img[i]), dtype=torch.float).min(),
        torch.tensor(np.array(img[i]), dtype=torch.float).mean(),
        torch.tensor(np.array(img[i]), dtype=torch.float).std())
# plt.imshow(img,  cmap='gray')
# plt.show()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [46]:

G.eval()

gen_img = G(torch.rand(1, 128).cuda(), torch.tensor([7]).cuda())
# mean = torch.as_tensor([0.5]*img.size(1), dtype=img.dtype, device=img.device).view(-1, 1, 1)
# std = torch.as_tensor([0.5]*img.size(1), dtype=img.dtype, device=img.device).view(-1, 1, 1)

m_img = gen_img.mul_(0.5).add_(0.5).mul_(255)
img = to_pil_image(gen_img[0])
img = np.array(img)

# img = gen_img

# print(img.min().item(), img.max().item(), img.mean().item(), img.std().item())
# # img = img.mul(255).add_(0.5).clamp_(0, 255).to("cpu", torch.uint8)

# plt.imshow(img[0].permute(1,2,0).detach().cpu().numpy(), cmap='gray')
# plt.show()

In [47]:
print(m_img.max(), img.max())

tensor(204.5060, device='cuda:0', grad_fn=<MaxBackward1>) 255


In [140]:
img, target = dataset_ori[0]
data = np.array(img)


dtype('uint8')

In [162]:
len(dataset)

10500

In [168]:
img, target = dataset[0]

img = np.array(img)
img.shape

(28, 28)

(256, 256, 3)

In [132]:
data_name = 'Places_LT'
data_type = 'train'
m = importlib.import_module('utils.datasets')
dataset = getattr(m, data_name)(is_train=(data_type == 'train'), is_extension=False)

img = Image.open(dataset.data[0])
img = np.array(img)
w, h, c = img.shape


In [133]:

with h5py.File('./data.h5py', 'a') as file:
    try:
        d_name = f'ori/{data_name}/{data_type}/data'
        data_dataset = file.create_dataset(d_name, shape=(len(dataset.data), w, h, c))
    except ValueError:
        del(file[d_name])
    try:
        t_name = f'ori/{data_name}/{data_type}/targets'
        targets_dataset = file.create_dataset(t_name, shape=(len(dataset.data),), dtype=int)
    except ValueError:
        del(file[t_name])

    print(file[d_name])
    print(file[t_name])


    for i, (img, target) in enumerate(zip(tqdm(dataset.data), dataset.targets)):
        img = Image.open(img)
        img = np.array(img)
        
        if img.ndim == 2:
            img = img[..., None].repeat(3, 2)
        
        data_dataset[i, :, :] = img
        targets_dataset[i] = target
      
    
    # imgs.append(img)
    # targets.append(target)

# print(mfile['ori']['FashionMNIST'].keys())
# imgs = []
# targets = []
# for img, target in dataset:
#     imgs.apend(img)
#     targets.apeend(target)




# try:
#     data = np.array(img)
#     group_name = f'ori/{data_name}/{data_type}/{target}'
#     mfile.create_dataset(group_name, data=data)
# except TypeError:
#     print(data.shape)
#     break
# mfile.close()

<HDF5 dataset "data": shape (62500, 256, 256, 3), type "<f4">
<HDF5 dataset "targets": shape (62500,), type "<i8">


 29%|██▊       | 17866/62500 [04:06<10:15, 72.51it/s] 


KeyboardInterrupt: 

ori
	CIFAR10_LT
		test
			data
			(10000, 32, 32, 3)
			targets
			(10000,)
		train
			data
			(9500, 32, 32, 3)
			targets
			(9500,)
	FashionMNIST_LT
		test
			data
			(10000, 28, 28)
			targets
			(10000,)
		train
			data
			(10500, 28, 28)
			targets
			(10500,)
	Places_LT
		test
			data
			(36500, 256, 256, 3)
			targets
			(36500,)
		train
			data
			(62500, 256, 256, 3)
			targets
			(62500,)


In [72]:
with h5py.File('./data.h5py', 'r') as file:
    # print_h5py(file)
    
    
    # file.create_dataset(t_name, shape=(len(dataset.data),), dtype=int)
    # del(file['ori/Places_LT/False'])
    # file['ori/Places_LT/test'] = file['ori/Places_LT/False']
    for n, v in file.items():
        print(n)
        # print(hasattr(n, "items"), hasattr(v, "items"))
        for n, v in v.items():
            print('\t'+n)
            for n, v in v.items():
                print('\t'+'\t'+n)
                for n, v in v.items():
                    # print(type(n), type(v))
                    print('\t'+'\t'+'\t'+n)
                    print('\t'+'\t'+'\t'+str(v.shape))
                    # print(f"{str(v.shape):30}")
                    # print("             ", v)
    
    # print(file['ori/Places_LT/train/data'][2000])
    # print(file['ori'].keys())
    # print(file['ori/CIFAR10_LT'].keys())
    # print(file[d_name])
    # print(file['ori']['FashionMNIST_LT']['train']['targets'][10000])


ori
	CIFAR10_LT
		test
			data
			(10000, 32, 32, 3)
			targets
			(10000,)
		train
			data
			(9500, 32, 32, 3)
			targets
			(9500,)
	FashionMNIST_LT
		test
			data
			(10000, 28, 28)
			targets
			(10000,)
		train
			data
			(10500, 28, 28)
			targets
			(10500,)
	Places_LT
		test
			data
			(36500, 256, 256, 3)
			targets
			(36500,)
		train
			data
			(62500, 256, 256, 3)
			targets
			(62500,)


In [39]:
for img, target in  zip(tqdm(dataset.data), dataset.targets):
    img = Image.open(img)
    img = np.array(img)
    
    print(type(img))
    print(type(target))

  1%|          | 114/10500 [00:00<00:11, 905.43it/s]

<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'nump

  3%|▎         | 295/10500 [00:00<00:12, 847.38it/s]

<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'nump

  4%|▍         | 470/10500 [00:00<00:13, 771.13it/s]

...

<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'nump

 12%|█▏        | 1230/10500 [00:01<00:09, 964.87it/s]

<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'nump

 12%|█▏        | 1271/10500 [00:01<00:10, 863.51it/s]


KeyboardInterrupt: 

 21%|██        | 13140/62500 [01:54<07:10, 114.78it/s]


KeyboardInterrupt: 

In [ ]:
np.concatenate(imgs).shape
np.concatenate(targets).shape

In [152]:
mfile = h5py.File('./data.h5py', 'a')
print(mfile['ori']['Places_LT']['train'].keys())
mfile.close()

<KeysViewHDF5 []>


In [18]:
np.unique(dataset_ori.targets, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [11]:
dataset_ori.num_classes.max() - dataset_ori.num_classes

array([4761, 4960, 4865, 4894, 4751, 4947, 4955,    0, 4872, 2368, 4947,
       4941, 2580, 4893, 4968, 4755, 4939, 4882, 4789, 3398, 4943, 4902,
       4932, 4973, 3450, 4924, 4887, 4896, 4962, 4797, 4825, 4908, 4888,
       4902, 4970, 4860, 4958, 4974, 4837, 4882, 4785, 4944, 3493, 4887,
       4907, 3574, 4972, 4940, 3600, 4842, 4969, 4966, 3895, 4937, 4921,
       4960, 4974, 4851, 4877, 4882, 4934, 4939, 3944, 4898, 4966, 4957,
       3957, 4139, 4972, 4889, 4960, 4950, 4961, 4946, 4936, 4969, 4782,
       4851, 4845, 4848, 4951, 4150, 4255, 4969, 4291, 4892, 4312, 4932,
       4975, 4942, 4337, 4375, 4909, 4974, 4414, 4908, 4946, 4433, 4926,
       4908, 4974, 4947, 4944, 4883, 4904, 4902, 4459, 4883, 4864, 4823,
       4481, 4972, 4881, 4906, 4956, 4918, 4952, 4852, 4804, 4908, 4893,
       4498, 4832, 4894, 4940, 4833, 4975, 4872, 4974, 4972, 4945, 4956,
       4500, 4948, 4840, 4973, 4896, 4974, 4934, 4958, 4876, 4895, 4862,
       4921, 4908, 4944, 4946, 4900, 4947, 4880, 45